In [11]:
import tensorflow as tf
from keras.datasets import mnist
import numpy as np
from PIL import ImageGrab, Image

In [3]:
# Load the EMNIST dataset (digits and uppercase letters)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape to include the channel dimension (1 for grayscale)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=36)  # 26 letters + 10 digits = 36 classes
y_test = tf.keras.utils.to_categorical(y_test, num_classes=36)


In [4]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(36, activation='softmax')  # 36 classes (26 letters + 10 digits)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))


Epoch 1/10
469/469 [==============================] - 17s 36ms/step - loss: 0.3918 - accuracy: 0.8839 - val_loss: 0.0917 - val_accuracy: 0.9722
Epoch 2/10
469/469 [==============================] - 17s 37ms/step - loss: 0.1289 - accuracy: 0.9615 - val_loss: 0.0674 - val_accuracy: 0.9783
Epoch 3/10
469/469 [==============================] - 28s 59ms/step - loss: 0.0961 - accuracy: 0.9717 - val_loss: 0.0516 - val_accuracy: 0.9827
Epoch 4/10
469/469 [==============================] - 28s 59ms/step - loss: 0.0777 - accuracy: 0.9762 - val_loss: 0.0506 - val_accuracy: 0.9838
Epoch 5/10
469/469 [==============================] - 27s 57ms/step - loss: 0.0687 - accuracy: 0.9789 - val_loss: 0.0465 - val_accuracy: 0.9845
Epoch 6/10
469/469 [==============================] - 27s 57ms/step - loss: 0.0582 - accuracy: 0.9822 - val_loss: 0.0398 - val_accuracy: 0.9858
Epoch 7/10
469/469 [==============================] - 27s 57ms/step - loss: 0.0532 - accuracy: 0.9839 - val_loss: 0.0385 - val_accuracy:

In [5]:
# Function to predict a single image
def predict_image(image):
    # Preprocess the image
    img = image.resize((28, 28))
    img = img.convert('L')
    img = np.array(img)
    img = img.reshape(1, 28, 28, 1)
    img = img / 255.0

    # Make prediction
    result = model.predict(img)[0]
    predicted_class = np.argmax(result)
    confidence = max(result)

    return predicted_class, confidence


In [6]:
# Test the model with a screenshot of the screen
def test_model_with_screenshot():
    rect = (0, 0, 300, 300)  # Define the region for the screenshot
    im = ImageGrab.grab(rect)
    predicted_class, confidence = predict_image(im)
    print(f"Predicted class: {predicted_class}, Confidence: {confidence*100:.2f}%")

# Test the model with a screenshot
test_model_with_screenshot()

1/1 [==============================] - 0s 168ms/step
Predicted class: 8, Confidence: 29.49%


In [9]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

# Load the pre-trained model
model = load_model('mnist.h5')

def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB to grayscale
    img = img.convert('L')
    img = np.array(img)
    # Reshaping to support our model input and normalizing
    img = img.reshape(1, 28, 28, 1)
    img = img / 255.0
    # Predict the class
    res = model.predict(img)[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Draw a digit", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognize", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, confidence = predict_digit(im)
        self.label.configure(text=f"Predicted digit: {digit}\nConfidence: {confidence*100:.2f}%")

    def clear_all(self):
        self.canvas.delete("all")

if __name__ == "__main__":
    app = App()
    mainloop()


1/1 [==============================] - 0s 47ms/step
